# Advent of Code 2024

> If debugging is the process of removing software bugs, then programming must be the process of putting them in.

-- Edsger W. Dijkstra

## Imports and definitions

In [1]:
import re
from itertools import chain
from urllib import request
from collections import Counter, defaultdict
from math import prod

def aocin(day):
    try:
        with open(f'input/{day}') as f:
            return f.read().strip()
    except FileNotFoundError:
        r = request.Request(f'https://adventofcode.com/2024/day/{day}/input')
        r.add_header('Cookie', open('../.aoccookie').read().strip())
        r.add_header('User-Agent', 'github.com/edoannunziata/jardin')
        with open(f'input/{day}', 'bw') as f:
            f.write(request.urlopen(r).read())
        with open(f'input/{day}') as f:
            return f.read().strip()

## [Day 1: Historian Hysteria](https://adventofcode.com/2024/day/1)

In [2]:
a, b = [list(t) for t in zip(*[[*map(int, x.split())] for x in aocin(1).split('\n')])]
na, nb = Counter(a), Counter(b)

A = sum(abs(u-v) for u, v in zip(sorted(a), sorted(b)))
assert A == 1388114

A = sum(u * nu * nb[u] for u, nu in na.items())
assert A == 23529853

## [Day 2: Red-Nosed Reports](https://adventofcode.com/2024/day/2)

In [3]:
reports = [[*map(int, l.split())] for l in aocin(2).split('\n')]

def is_safe(l, allowed, tol=0):
    def _f(l, tol):
        if tol < 0: return False
        if len(l) < 2: return True
        a, b, *rest = l
        return (
            ((a is None or a-b in allowed) and _f([b] + rest, tol))
            or _f([a] + rest, tol-1)
        )
    return _f([None] + l, tol)


A = sum(is_safe(l, {1, 2, 3}) or is_safe(l, {-1, -2, -3}) for l in reports)
assert A == 369

A = sum(is_safe(l, {1, 2, 3}, tol=1) or is_safe(l, {-1, -2, -3}, tol=1) for l in reports)
assert A == 428

## [Day 3: Mull it Over](https://adventofcode.com/2024/day/3)

In [4]:
A = sum(
    prod(map(int, m.groups()))
    for m in re.finditer(r'mul\((\d{1,3}),(\d{1,3})\)', aocin(3))
)
assert A == 160672468


def mul_with_switch(input):
    active, acc = True, 0
    for m in re.finditer(r"(do|don't|mul)\((\d{0,3}),?(\d{0,3})\)", input):
        match m.groups():
            case "mul", _ as a, _ as b:
                acc += active * int(a) * int(b)
            case "do", _, _:
                active = True
            case "don't", _, _:
                active = False
    return acc

A = mul_with_switch(aocin(3))
assert A == 84893551

## [Day 4: Ceres Search](https://adventofcode.com/2024/day/4)

In [5]:
def gen_input(raw):
    rows = raw.split('\n')
    cols = [''.join(u) for u in list(zip(*rows))]
    diag_primary = defaultdict(str)
    diag_secondary = defaultdict(str)
    for i in range(len(rows)):
        for j in range(len(rows[0])):
            diag_primary[i-j] += rows[i][j]
            diag_secondary[i+j] += rows[i][j]
    return (
        rows, cols, diag_primary.values(), diag_secondary.values()
    )


rows, cols, diag_primary, diag_secondary = gen_input(aocin(4))
A = sum(
    u.count('XMAS') + u.count('SAMX')
    for u in chain(rows, cols, diag_primary, diag_secondary)
)
assert A == 2613

A = sum(
    (
            rows[i][j] == 'A'
        and {rows[i+1][j+1], rows[i-1][j-1]} == {'S', 'M'}
        and {rows[i+1][j-1], rows[i-1][j+1]} == {'S', 'M'} 
    )
    for i in range(1, len(rows) - 1)
    for j in range(1, len(rows[0]) - 1)
)
assert A == 1905